## Natural Language Process

### N-Gram

In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F

In [2]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # whether GPU is supportted
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [3]:
trigram = [((test_sentence[i], test_sentence[i+1]), test_sentence[i+2]) for i in range(len(test_sentence)-2)]
vocb = set(test_sentence)
word_to_idx = {word: i for i, word in enumerate(vocb)}
idx_to_word = {word_to_idx[word]: word for word in word_to_idx}

In [4]:
class NgramModel(nn.Module):
    def __init__(self, vocb_size, context_size, n_dim):
        super(NgramModel, self).__init__()
        self.n_word = vocb_size
        self.embedding = nn.Embedding(self.n_word, n_dim)
        self.linear1 = nn.Linear(context_size * n_dim, 128)
        self.linear2 = nn.Linear(128, self.n_word)
        
    def forward(self, x):
        emb = self.embedding(x)
        emb = emb.view(1, -1)
        out = self.linear1(emb)
        out = F.relu(out)
        out = self.linear2(out)
        log_prob = F.log_softmax(out, dim=1)
        return log_prob

In [5]:
ngrammodel = NgramModel(len(word_to_idx), CONTEXT_SIZE, 100).to(device)

In [6]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(ngrammodel.parameters(), lr=1e-3)

In [7]:
for epoch in range(100):
    print(f'epoch: {epoch+1}')
    print('*' * 10)
    running_loss =0
    for data in trigram:
        word, label = data
        word = torch.LongTensor([word_to_idx[i] for i in word]).to(device)
        label = torch.LongTensor([word_to_idx[label]]).to(device)
        # forward
        out = ngrammodel(word)
        loss = criterion(out, label)
        running_loss += loss
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Loss: {running_loss/len(word_to_idx):.6f}')

epoch: 1
**********
Loss: 5.382554
epoch: 2
**********
Loss: 5.322231
epoch: 3
**********
Loss: 5.262753
epoch: 4
**********
Loss: 5.203866
epoch: 5
**********
Loss: 5.145464
epoch: 6
**********
Loss: 5.087427
epoch: 7
**********
Loss: 5.029656
epoch: 8
**********
Loss: 4.972167
epoch: 9
**********
Loss: 4.914948
epoch: 10
**********
Loss: 4.857956
epoch: 11
**********
Loss: 4.801033
epoch: 12
**********
Loss: 4.743916
epoch: 13
**********
Loss: 4.686612
epoch: 14
**********
Loss: 4.629117
epoch: 15
**********
Loss: 4.571394
epoch: 16
**********
Loss: 4.513298
epoch: 17
**********
Loss: 4.454772
epoch: 18
**********
Loss: 4.395670
epoch: 19
**********
Loss: 4.336003
epoch: 20
**********
Loss: 4.275900
epoch: 21
**********
Loss: 4.215404
epoch: 22
**********
Loss: 4.154387
epoch: 23
**********
Loss: 4.092783
epoch: 24
**********
Loss: 4.030643
epoch: 25
**********
Loss: 3.967951
epoch: 26
**********
Loss: 3.904687
epoch: 27
**********
Loss: 3.841022
epoch: 28
**********
Loss: 3.776984
e

In [8]:
word, label = trigram[3]
word = torch.LongTensor([word_to_idx[i] for i in word]).to(device)
out = ngrammodel(word)
_, predict_label = torch.max(out, 1)
predict_word = idx_to_word[predict_label.item()]
print(f'real word is {label}, predict word is {predict_word}')

real word is thy, predict word is thy
